In [230]:
import pandas as pd
from coptpy import COPT
from gurobipy import GRB
import numpy as np
from template_generate import *
import utils as utils
from config.network import construct_network
from config.param import Param
from dnp_model import DNP
from ncg.np_cg import *
from csp.cp_gfc import *
import pickle
import folium

In [231]:
param = Param()
arg = param.arg
arg.backorder_sku_unit_cost = 5000
arg.capacity_node_ratio = 1
arg.capacity_ratio = 1
arg.cardinality_limit = 30
arg.distance_limit = 5000
arg.holding_sku_unit_cost = 1
arg.in_upper_ratio = 0.24    # 2/2.8的时候LP=MIP
arg.lb_end_ratio = 1
arg.lb_inter_ratio = 1
arg.node_lb_ratio = 1
arg.in_upper_qty = 100
arg.unfulfill_sku_unit_cost = 5000
arg.conf_label = 3
arg.pick_instance = 2
arg.backorder = 0
arg.transportation_sku_unit_cost = 1
arg.T = 1
arg.terminate_condition = 1e-5
arg.new_data = True
arg.num_periods = 20
arg.add_in_upper = False
utils.configuration(arg.conf_label, arg)
print(
    json.dumps(
        arg.__dict__,
        indent=2,
        sort_keys=True,
    )
)
arg.new_data = 1
arg.num_periods = 20
arg.terminate_condition = 1e-4
datapath = '/home/chuwen/cora/facility-loc-inventory/data/small_instance_generate_2/'
arg.path = datapath

{
  "T": 1,
  "add_cardinality": 0,
  "add_distance": 0,
  "add_in_upper": 1,
  "backend": "gurobi",
  "backorder": 0,
  "backorder_sku_unit_cost": 5000,
  "bool_use_ncg": 1,
  "capacity": 1,
  "capacity_node_ratio": 1,
  "capacity_ratio": 1,
  "cardinality": 0,
  "cardinality_limit": 30,
  "cg_itermax": 10,
  "cg_method_mip_heuristic": -1,
  "cg_mip_recover": 1,
  "cg_rmp_mip_iter": 10,
  "check_cost_cg": 0,
  "conf_label": 3,
  "covering": 1,
  "cus_num": 4,
  "d": 150,
  "del_col_freq": 3,
  "del_col_stra": 1,
  "demand_type": 1,
  "distance": 0,
  "distance_limit": 5000,
  "edge_lb": 1,
  "holding_sku_unit_cost": 1,
  "if_del_col": null,
  "if_fixed_cost": 0,
  "in_upper_qty": 100,
  "in_upper_ratio": 0.24,
  "lb_end_ratio": 1,
  "lb_inter_ratio": 1,
  "new_data": true,
  "node_lb": 0,
  "node_lb_ratio": 1,
  "num_periods": 20,
  "num_skus": 500,
  "path": "data/data_0401_0inv.xlsx",
  "pick_instance": 2,
  "plant_fixed_cost": 200,
  "pricing_relaxation": 0,
  "production_sku_unit_

In [232]:
def check_solution(name,model):
    os.makedirs(f"sol_{arg.pick_instance}_"+name, exist_ok=True)
    model.get_solution(f"sol_{arg.pick_instance}_"+name)
    df_edges = pd.read_csv(f"sol_{arg.pick_instance}_"+name+"edge_sku_t_flow.csv")
    if df_edges.empty:
        print("No solution")
    else:
        print('----------Originial Solution----------')
        for _,row in df_edges.iterrows():
            if 0 < row['y'] <1:
                print("Fractional edges",row["obj_start"], row["obj_end"], row["t"],row['y'])
            else:
                print("Integeral edges",row["obj_start"], row["obj_end"], row["t"],row['y'])
    print('------Cost Information--------')
    print('holding_cost',model.obj['holding_cost'][0].getExpr().getValue())
    print('transportation_cost',model.obj['transportation_cost'].getExpr().getValue())
    print('unfulfilled_demand_cost',model.obj['unfulfilled_demand_cost'][0].getExpr().getValue())
    

In [233]:
dnp_mps_name = f"new_guro_{datapath.split('/')[1]}_{arg.T}_{arg.conf_label}@{arg.pick_instance}@{arg.backorder}.mps"
(
        sku_list,
        plant_list,
        warehouse_list,
        customer_list,
        edge_list,
        network,
        node_list,
        *_,
) = utils.scale(arg.pick_instance, datapath, arg)
utils.add_attr(edge_list, node_list, arg, const)
network = construct_network(node_list, edge_list, sku_list)
solver = arg.backend.upper()
arg.DNP = 1
arg.sku_list = sku_list
model = DNP(arg, network)
model.modeling()
model.model.setParam("Logging", 1)
model.model.setParam("Threads", 8)
model.model.setParam("TimeLimit", 7200)
model.model.setParam("LpMethod", 2)
model.model.setParam("Crossover", 0)
print(f"save mps name {dnp_mps_name}")
model.model.write(dnp_mps_name)
print("-----------MIP------------")
model.solve()
mipval = model.get_model_objval()
model.write('data/model_mip.lp')
check_solution("MIP/",model)

[facinv] 2024-04-24 21:10:24,847: time scale 1
[facinv] 2024-04-24 21:10:24,848: generating the signature of this problem
[facinv] 2024-04-24 21:10:24,848: {
  "customer_num": 4,
  "data_dir": "/home/chuwen/cora/facility-loc-inventory/data/small_instance_generate_2/",
  "one_period": true,
  "plant_num": 20,
  "sku_num": 10,
  "warehouse_num": 20
}
[facinv] 2024-04-24 21:10:24,848: current data has been generated before
[facinv] 2024-04-24 21:10:24,849: reading from cache: ./out/home-4-True-20-10-20.pk
[facinv] 2024-04-24 21:10:24,911: dumping a temporary cache @./out/home-4-True-20-10-20.pk
[facinv] 2024-04-24 21:10:24,912: setting arg.sku_list...
Cardinal Optimizer v7.0.6. Build date Jan 24 2024
Copyright Cardinal Operations 2023. All Rights Reserved

add_in_uppper T6 24840.0
add_in_uppper T12 23400.0
add_in_uppper T13 22680.0
Edge Cost E(P1, T12) 17.877261698362073
Edge Cost E(T6, C1) 231.16916933851738
Edge Cost E(T12, T13) 0.0
Edge Cost E(T12, T6) 638.9016328026747
Edge Cost E(T13

In [234]:
print("-----------LP------------")
for k, v in model.variables['select_edge'].items():
    v.setType(COPT.CONTINUOUS)
    print(f"relaxing {k}")
for k, v in model.variables['sku_select_edge'].items():
    v.setType(COPT.CONTINUOUS)
    print(f"relaxing {k}")
model.solve()
lpval = model.get_model_objval()
model.write('data/model_lp.lp')
print('LP SOLUTION',lpval)
check_solution("LP/",model)
lp_model = model

-----------LP------------
relaxing (0, E(P1, T12))
relaxing (0, E(T6, C1))
relaxing (0, E(T12, T13))
relaxing (0, E(T12, T6))
relaxing (0, E(T13, C1))
relaxing (0, E(P1, T12), SKU_SKU0002)
relaxing (0, E(T6, C1), SKU_SKU0002)
relaxing (0, E(T12, T13), SKU_SKU0002)
relaxing (0, E(T12, T6), SKU_SKU0002)
relaxing (0, E(T13, C1), SKU_SKU0002)
Model fingerprint: 12ce5729

Using Cardinal Optimizer v7.0.6 on Linux
Hardware has 32 cores and 64 threads. Using instruction set X86_AVX512_E1 (14)
Minimizing a MIP problem

The original problem has:
    44 rows, 26 columns and 89 non-zero elements
    6 binaries

Starting the MIP solver with 8 threads and 32 tasks

Presolving the problem
Problem was solved during presolve

Best solution   : 1787.726169836
Best bound      : 1787.726169836
Best gap        : 0.0000%
Solve time      : 0.01
Work count      : 0.0000423378000000
Solve node      : 0
MIP status      : solved
Solution status : integer optimal (relative gap limit 0.0001)

Violations      :    

In [235]:
def enumerate_subsets( N1, N2, d, dump=False, verbose=False):
    feasible_set = {}
    subset_of_N1 = generate_subsets(N1)
    for c1 in subset_of_N1:
        N1UN2 = list(set(N1) | set(N2))
        R_set = generate_subsets(N1UN2)
        for r in R_set:
            A = set(c1) - set(r)
            B = set(c1) & set(r)
            lbd = 0
            for i in A:
                lbd += i.capacity
            for j in B:
                lbd += j.variable_lb
            lbd = lbd - d
            if lbd > 0:
                # subsets = (tuple(A), tuple(B))
                subsets = (tuple(c1), tuple(r))
                feasible_set[subsets] = lbd
    return feasible_set

def generate_subsets(nums):
    def backtrack(start, subset):
        result.append(subset[:])
        for i in range(start, len(nums)):
            subset.append(nums[i])
            backtrack(i + 1, subset)
            subset.pop()

    result = []
    backtrack(0, [])
    return result

def query_fractional_inter_edges(model):
    fractional_edge_list = []
    for e in network.edges:
        edge = network.edges[e]["object"]
        t = 0
        variable_y = model.variables["select_edge"][t, edge].x
        if variable_y > 0 and variable_y < 1:
            fractional_edge_list.append(edge)    
    return len(fractional_edge_list)>0,fractional_edge_list

In [236]:
K = 20
k = 0
t = 0
# choose = 'Proposition 1'
# choose = 'Proposition 2'
# choose = 'Corollary 3'
# choose = 'Corollary 4'
choose = 'Corollary 5'
if choose == 'Proposition 1':
    model_1 = lp_model
    exist_fractional_edge, fractional_edge_list = query_fractional_inter_edges(model_1)
    while exist_fractional_edge and k < K:
        print(f'-----------Iteration {k}------------')
        print('Fractional edges',fractional_edge_list)
        for edge in fractional_edge_list:
            start, node = edge.start, edge.end
            if node.type == 'C':
                d = sum(edge.end.demand[t])
            else:
                d = arg.in_upper_qty
            N1 = [e for e in utils.get_in_edges(network, node)]
            N2 = []
            print('For the fractional edge',edge,'the inflow edges (set N1) are',N1)
            subset_of_N1 = generate_subsets(N1)
            for c1 in subset_of_N1:
                A = list(c1)
                lbd = sum(i.capacity for i in A) - d
                if lbd > 0:
                    u_hat = max(max(i.capacity for i in c1),lbd)
                    subset_of_L1 = generate_subsets(list(set(N1)-set(c1)))
                    n = 0
                    for l1 in subset_of_L1:
                        B = list(l1)
                        lhs = 0
                        for j in A:
                            lhs += model_1.variables["sku_flow"].sum(t,j,'*')+max(j.capacity-lbd,0)*(1-model_1.variables["select_edge"][t,j])
                        for j in B:
                            lhs += model_1.variables["sku_flow"].sum(t,j,'*')-(max(u_hat,j.capacity)-lbd)*model_1.variables["select_edge"][t,j]
                        model_1.model.addConstr(lhs <= d)
                        model_1.model.solve()
                        print("NODE",node,"N",n,"K",k)
                        check_solution(f"LP_propo1_{n}/",model_1)
                        n += 1
            k += 1
            exist_fractional_edge, fractional_edge_list = query_fractional_inter_edges(model_1)
elif choose == 'Proposition 2':
    model_2 = lp_model
    exist_fractional_edge, fractional_edge_list = query_fractional_inter_edges(model_2)
    while exist_fractional_edge and k < K:
        print(f'-----------Iteration {k}------------')
        print('Fractional edges',fractional_edge_list)
        for edge in fractional_edge_list:
                start, node = edge.start, edge.end
                if node.type == 'C':
                    d = sum(edge.end.demand[t])
                else:
                    d = arg.in_upper_qty
                N1 = [e for e in utils.get_in_edges(network, node)]
                N2 = []
                print('For the fractional edge',edge,'the inflow edges (set N1) are',N1)
                feasible_set = enumerate_subsets(N1, N2, d, dump=False, verbose=False)
                n = 0
                for set_candidate,lbd in feasible_set.items():
                    c1 = list(set_candidate[0])
                    r = list(set_candidate[1])
                    c1dr = list(set(c1) - set(r))
                    c1ir = list(set(c1) & set(r))
                    u_hat = max(max(i.capacity for i in c1dr) if len(c1dr)>0 else 0,max(i.variable_lb for i in c1ir) if len(c1ir)>0 else 0,lbd)
                    L1 = generate_subsets(list(set(N1)-set(c1)))
                    for l1 in L1:
                        l1dr = list(set(l1) - set(r))
                        l1ir = list(set(l1) & set(r))
                        lhs = 0
                        for j in c1ir:
                            lhs = lhs + model_2.variables["select_edge"][t,j]*j.variable_lb+max(j.variable_lb-lbd,0)*(1-model_2.variables["select_edge"][t,j])
                        for j in c1dr:
                            lhs = lhs + model_2.variables["sku_flow"].sum(t,j,'*')+max(j.capacity-lbd,0)*(1-model_2.variables["select_edge"][t,j])
                        for j in l1ir:
                            lhs = lhs + (j.variable_lb-(max(u_hat,j.variable_lb)-lbd))*model_2.variables["select_edge"][t,j]
                        for j in l1dr:
                            lhs = lhs + model_2.variables["sku_flow"].sum(t,j,'*') - (max(u_hat,j.capacity)-lbd)*model_2.variables["select_edge"][t,j]
                        print('lhs',lhs,'c1dr',c1dr,'c1ir',c1ir,'l1dr',l1dr,'l1ir',l1ir,'u_hat',u_hat,'lbd',lbd)
                        print("NODE",node,"N",n,"K",k)
                        model_2.model.addConstr(lhs <= d)
                        model_2.model.solve()
                        if model_2.model.status == COPT.INFEASIBLE:
                            model_2.model.computeIIS()
                            model_2.model.writeIIS(f'data/model2_{edge}_{n}_{c1}.iis')
                        else:
                            check_solution(f"LP_propo2_{n}/",model_2)
                        n += 1
                    k += 1
                    exist_fractional_edge, fractional_edge_list = query_fractional_inter_edges(model_2)

elif choose == 'Corollary 3':
    model_3 = lp_model
    exist_fractional_edge, fractional_edge_list = query_fractional_inter_edges(model_3)
    while exist_fractional_edge and k < K:
        print(f'-----------Iteration {k}------------')
        print('Fractional edges',fractional_edge_list)
        for edge in fractional_edge_list:
            start, node = edge.start, edge.end
            if node.type == 'C':
                d = sum(edge.end.demand[t])
            else:
                d = arg.in_upper_qty
            N1 = [e for e in utils.get_in_edges(network, node)]
            N2 = []
            print('For the fractional edge',edge,'the inflow edges (set N1) are',N1)
            feasible_set = enumerate_subsets(N1, N2, d, dump=False, verbose=False)
            n = 0
            for set_candidate,lbd in feasible_set.items():
                c1 = list(set_candidate[0])
                r = list(set_candidate[1])
                c1dr = list(set(c1) - set(r))
                c1ir = list(set(c1) & set(r))
                lhs = 0
                for j in c1dr:
                    lhs += model_3.variables["sku_flow"].sum(t,j,'*')+max(j.capacity-lbd,0)*(1-model_3.variables["select_edge"][t,j])
                for j in c1ir:
                    lhs += max(j.variable_lb-lbd,0)+min(j.variable_lb,lbd)*(model_3.variables["select_edge"][t,j])
                model_3.model.addConstr(lhs <= d)
                model_3.model.solve()
                print(f'-----For node {node} with d={d}, we add the {n}th valid inequality in C1\R {c1dr} and C1&R {c1ir}------------')
                check_solution(f"LP_collary3_{n}/",model_3)
                n += 1
        k += 1
        exist_fractional_edge, fractional_edge_list = query_fractional_inter_edges(model_3)
elif choose == 'Corollary 4':
    model_4 = lp_model
    exist_fractional_edge, fractional_edge_list = query_fractional_inter_edges(model_4)
    while exist_fractional_edge and k < K:
        print(f'-----------Iteration {k}------------')
        print('Fractional edges',fractional_edge_list)
        for edge in fractional_edge_list:
            start, node = edge.start, edge.end
            if node.type == 'C':
                d = sum(edge.end.demand[t])
            else:
                d = arg.in_upper_qty
            N1 = [e for e in utils.get_in_edges(network, node)]
            N2 = []
            print('For the fractional edge',edge,'the inflow edges (set N1) are',N1)
            n = 0
            subset_of_N1 = generate_subsets(N1)
            for c1 in subset_of_N1:
                A = list(c1)
                lbd = sum(i.capacity for i in A) - d
                if lbd > 0:
                    lhs = 0
                    for j in A:
                        lhs = lhs + model_4.variables["sku_flow"].sum(t,j,'*') + max(j.capacity-lbd,0)*(1-model_4.variables["select_edge"][t,j])
                    print('lhs',lhs)
                    model_4.model.addConstr(lhs <= d)
                    model_4.model.solve()
                    model_4.write(f'data/model_{edge}_{n}_{c1}.lp')
                    print(f'-----For node {node} with d={d}, we add the {n}th valid inequality in C1{A}------------')
                    check_solution(f"LP_collary4_{n}/",model_4)
                    n += 1
        k += 1
        exist_fractional_edge, fractional_edge_list = query_fractional_inter_edges(model_4)
elif choose == 'Corollary 5':
    model_5 = lp_model
    exist_fractional_edge, fractional_edge_list = query_fractional_inter_edges(model_5)
    while exist_fractional_edge and k < K:
        print(f'-----------Iteration {k}------------')
        print('Fractional edges',fractional_edge_list)
        for edge in fractional_edge_list:
            start, node = edge.start, edge.end
            if node.type == 'C':
                d = sum(edge.end.demand[t])
            else:
                d = arg.in_upper_qty
            N1 = [e for e in utils.get_in_edges(network, node)]
            N2 = []
            print('For the fractional edge',edge,'the inflow edges (set N1) are',N1)
            subset_of_N1 = generate_subsets(N1)
            for c1 in subset_of_N1:
                n = 0
                A = list(c1)
                if len(A)>0 :
                    lbd = sum(i.capacity for i in A) - d
                    u_hat = max(i.capacity for i in A)
                    if lbd > 0 and u_hat > lbd:
                        L1 = generate_subsets(list(set(N1)-set(c1)))
                        for l1 in L1:
                            lhs = 0
                            B = list(l1)
                            for j in A:
                                lhs += model_5.variables["sku_flow"].sum(t,j,'*')+max(j.capacity-lbd,0)*(1-model_5.variables["select_edge"][t,j])
                            for j in B:
                                lhs += model_5.variables["sku_flow"].sum(t,j,'*')-(max(u_hat,j.capacity)-lbd)*model_5.variables["select_edge"][t,j]
                        model_5.model.addConstr(lhs <= d)
                        model_5.model.solve()
                        print(f'-----For node {node} with d={d}, we add the {n}th valid inequality in C1 {A} and L1 {B}------------')
                        check_solution(f"LP_collary5_{n}/",model_5)
                        n += 1
        k += 1
        exist_fractional_edge, fractional_edge_list = query_fractional_inter_edges(model_5)

-----------Iteration 0------------
Fractional edges [E(P1, T12), E(T12, T13), E(T13, C1)]
For the fractional edge E(P1, T12) the inflow edges (set N1) are [E(P1, T12)]
Model fingerprint: 39193991

Using Cardinal Optimizer v7.0.6 on Linux
Hardware has 32 cores and 64 threads. Using instruction set X86_AVX512_E1 (14)
Minimizing a MIP problem

The original problem has:
    45 rows, 26 columns and 91 non-zero elements
    6 binaries

Starting the MIP solver with 8 threads and 32 tasks

Presolving the problem
Problem was solved during presolve

Best solution   : 1787.726169836
Best bound      : 1787.726169836
Best gap        : 0.0000%
Solve time      : 0.01
Work count      : 0.0000425026000000
Solve node      : 0
MIP status      : solved
Solution status : integer optimal (relative gap limit 0.0001)

Violations      :     absolute     relative
  bounds        :            0            0
  rows          :  1.42109e-14  1.42109e-14
  integrality   :            0
-----For node T12 with d=100, w

#### Proposition 1.

In [237]:
# model_1 = lp_model
# exist_fractional_edge, fractional_edge_list = query_fractional_inter_edges(model_1)
# while exist_fractional_edge and k < K:
#     print(f'-----------Iteration {k}------------')
#     print('Fractional edges',fractional_edge_list)
#     for edge in fractional_edge_list:
#         start, node = edge.start, edge.end
#         if node.type == 'C':
#             d = sum(edge.end.demand[t])
#         else:
#             d = arg.in_upper_qty
#         N1 = [e for e in utils.get_in_edges(network, node)]
#         N2 = []
#         print('For the fractional edge',edge,'the inflow edges (set N1) are',N1)
#         subset_of_N1 = generate_subsets(N1)
#         for c1 in subset_of_N1:
#             A = list(c1)
#             lbd = sum(i.capacity for i in A) - d
#             if lbd > 0:
#                 u_hat = max(max(i.capacity for i in c1),lbd)
#                 subset_of_L1 = generate_subsets(list(set(N1)-set(c1)))
#                 n = 0
#                 for l1 in subset_of_L1:
#                     B = list(l1)
#                     lhs = 0
#                     for j in A:
#                         lhs += model_1.variables["sku_flow"].sum(t,j,'*')+max(j.capacity-lbd,0)*(1-model_1.variables["select_edge"][t,j])
#                     for j in B:
#                         lhs += model_1.variables["sku_flow"].sum(t,j,'*')-(max(u_hat,j.capacity)-lbd)*model_1.variables["select_edge"][t,j]
#                     model_1.model.addConstr(lhs <= d)
#                     model_1.model.solve()
#                     print("NODE",node,"N",n,"K",k)
#                     check_solution(f"LP_propo1_{n}/",model_1)
#                     n += 1
#         k += 1
#         exist_fractional_edge, fractional_edge_list = query_fractional_inter_edges(model_1)

#### Proposition 2

In [238]:
# K = 20
# k = 0
# t = 0
# model_2 = lp_model
# exist_fractional_edge, fractional_edge_list = query_fractional_inter_edges(model_2)
# while exist_fractional_edge and k < K:
#     print(f'-----------Iteration {k}------------')
#     print('Fractional edges',fractional_edge_list)
#     for edge in fractional_edge_list:
#             start, node = edge.start, edge.end
#             if node.type == 'C':
#                 d = sum(edge.end.demand[t])
#             else:
#                 d = arg.in_upper_qty
#             N1 = [e for e in utils.get_in_edges(network, node)]
#             N2 = []
#             print('For the fractional edge',edge,'the inflow edges (set N1) are',N1)
#             feasible_set = enumerate_subsets(N1, N2, d, dump=False, verbose=False)
#             n = 0
#             for set_candidate,lbd in feasible_set.items():
#                 c1 = list(set_candidate[0])
#                 r = list(set_candidate[1])
#                 c1dr = list(set(c1) - set(r))
#                 c1ir = list(set(c1) & set(r))
#                 u_hat = max(max(i.capacity for i in c1dr) if len(c1dr)>0 else 0,max(i.variable_lb for i in c1ir) if len(c1ir)>0 else 0,lbd)
#                 L1 = generate_subsets(list(set(N1)-set(c1)))
#                 for l1 in L1:
#                     l1dr = list(set(l1) - set(r))
#                     l1ir = list(set(l1) & set(r))
#                     lhs = 0
#                     for j in c1ir:
#                         lhs = lhs + model_2.variables["select_edge"][t,j]*j.variable_lb+max(j.variable_lb-lbd,0)*(1-model_2.variables["select_edge"][t,j])
#                     for j in c1dr:
#                         lhs = lhs + model_2.variables["sku_flow"].sum(t,j,'*')+max(j.capacity-lbd,0)*(1-model_2.variables["select_edge"][t,j])
#                     for j in l1ir:
#                         lhs = lhs + (j.variable_lb-(max(u_hat,j.variable_lb)-lbd))*model_2.variables["select_edge"][t,j]
#                     for j in l1dr:
#                         lhs = lhs + model_2.variables["sku_flow"].sum(t,j,'*') - (max(u_hat,j.capacity)-lbd)*model_2.variables["select_edge"][t,j]
#                     print('lhs',lhs,'c1dr',c1dr,'c1ir',c1ir,'l1dr',l1dr,'l1ir',l1ir,'u_hat',u_hat,'lbd',lbd)
#                     print("NODE",node,"N",n,"K",k)
#                     model_2.model.addConstr(lhs <= d)
#                     model_2.model.solve()
#                     if model_2.model.status == COPT.INFEASIBLE:
#                         model_2.model.computeIIS()
#                         model_2.model.writeIIS(f'data/model2_{edge}_{n}_{c1}.iis')
#                     else:
#                         check_solution(f"LP_propo2_{n}/",model_2)
#                     n += 1
#                 k += 1
#                 exist_fractional_edge, fractional_edge_list = query_fractional_inter_edges(model_2)


#### Corollary 3

In [239]:
# model_3 = lp_model

In [240]:
# t = 0
# k = 0
# K = 3
# exist_fractional_edge, fractional_edge_list = query_fractional_inter_edges(model_3)
# while exist_fractional_edge and k < K:
#     print(f'-----------Iteration {k}------------')
#     print('Fractional edges',fractional_edge_list)
#     for edge in fractional_edge_list:
#         start, node = edge.start, edge.end
#         if node.type == 'C':
#             d = sum(edge.end.demand[t])
#         else:
#             d = arg.in_upper_qty
#         N1 = [e for e in utils.get_in_edges(network, node)]
#         N2 = []
#         print('For the fractional edge',edge,'the inflow edges (set N1) are',N1)
#         feasible_set = enumerate_subsets(N1, N2, d, dump=False, verbose=False)
#         n = 0
#         for set_candidate,lbd in feasible_set.items():
#             A = list(set_candidate[0])
#             B = list(set_candidate[1])
#             lhs = 0
#             for j in A:
#                 lhs += model_3.variables["sku_flow"].sum(t,j,'*')+max(j.capacity-lbd,0)*(1-model_3.variables["select_edge"][t,j])
#             for j in B:
#                 lhs += max(j.variable_lb-lbd,0)+min(j.variable_lb,lbd)*(model_3.variables["select_edge"][t,j])
#             model_3.model.addConstr(lhs <= d)
#             model_3.model.solve()
#             print(f'-----For node {node} with d={d}, we add the {n}th valid inequality in C1\R {A} and C1&R {B}------------')
#             check_solution(f"LP_collary3_{n}/",model_3)
#             n += 1
#     k += 1
#     exist_fractional_edge, fractional_edge_list = query_fractional_inter_edges(model_3)


#### Corollary 4


In [241]:
# model_4 = lp_model

In [242]:
# t = 0
# k = 0
# K = 3
# exist_fractional_edge, fractional_edge_list = query_fractional_inter_edges(model_4)
# while exist_fractional_edge and k < K:
#     print(f'-----------Iteration {k}------------')
#     print('Fractional edges',fractional_edge_list)
#     for edge in fractional_edge_list:
#         start, node = edge.start, edge.end
#         if node.type == 'C':
#             d = sum(edge.end.demand[t])
#         else:
#             d = arg.in_upper_qty
#         N1 = [e for e in utils.get_in_edges(network, node)]
#         N2 = []
#         print('For the fractional edge',edge,'the inflow edges (set N1) are',N1)
#         n = 0
#         subset_of_N1 = generate_subsets(N1)
#         for c1 in subset_of_N1:
#             A = list(c1)
#             lbd = sum(i.capacity for i in A) - d
#             if lbd > 0:
#                 lhs = 0
#                 for j in A:
#                     lhs = lhs + model_4.variables["sku_flow"].sum(t,j,'*') + max(j.capacity-lbd,0)*(1-model_4.variables["select_edge"][t,j])
#                 print('lhs',lhs)
#                 model_4.model.addConstr(lhs <= d)
#                 model_4.model.solve()
#                 model_4.write(f'data/model_{edge}_{n}_{c1}.lp')
#                 print(f'-----For node {node} with d={d}, we add the {n}th valid inequality in C1{A}------------')
#                 check_solution(f"LP_collary4_{n}/",model_4)
#                 n += 1
#     k += 1
#     exist_fractional_edge, fractional_edge_list = query_fractional_inter_edges(model_4)

#### Corollary 5

In [243]:
# model_5 = lp_model

In [244]:
# t = 0
# k = 0
# K = 3
# exist_fractional_edge, fractional_edge_list = query_fractional_inter_edges(model_5)
# while exist_fractional_edge and k < K:
#     print(f'-----------Iteration {k}------------')
#     print('Fractional edges',fractional_edge_list)
#     for edge in fractional_edge_list:
#         start, node = edge.start, edge.end
#         if node.type == 'C':
#             d = sum(edge.end.demand[t])
#         else:
#             d = arg.in_upper_qty
#         N1 = [e for e in utils.get_in_edges(network, node)]
#         N2 = []
#         print('For the fractional edge',edge,'the inflow edges (set N1) are',N1)
#         subset_of_N1 = generate_subsets(N1)
#         for c1 in subset_of_N1:
#             n = 0
#             A = list(c1)
#             if len(A)>0 :
#                 lbd = sum(i.capacity for i in A) - d
#                 u_hat = max(i.capacity for i in A)
#                 if lbd > 0 and u_hat > lbd:
#                     print('lbd',lbd,'u_hat',u_hat)
#                     L1 = generate_subsets(list(set(N1)-set(c1)))
#                     for l1 in L1:
#                         lhs = 0
#                         B = list(l1)
#                         for j in A:
#                             lhs += model_5.variables["sku_flow"].sum(t,j,'*')+max(j.capacity-lbd,0)*(1-model_5.variables["select_edge"][t,j])
#                         for j in B:
#                             lhs += model_5.variables["sku_flow"].sum(t,j,'*')-(max(u_hat,j.capacity)-lbd)*model_5.variables["select_edge"][t,j]
#                         print(lhs)
#                         print('C1',A,'L1',B)
#                         model_5.model.addConstr(lhs <= d)
#                         model_5.model.solve()
#                         model_5.write(f'data/model5_{edge}_{n}_{c1}.lp')
#                         if model_5.model.status == COPT.INFEASIBLE:
#                             model_5.model.computeIIS()
#                             model_5.model.writeIIS(f'data/model5_{edge}_{n}_{c1}.iis')
#                         print(f'-----For node {node} with d={d}, we add the {n}th valid inequality in C1 {A} and L1 {B}------------')
#                         check_solution(f"LP_collary5_{n}/",model_5)
#                         n += 1
#     k += 1
#     exist_fractional_edge, fractional_edge_list = query_fractional_inter_edges(model_5)
        

In [245]:
# pickle.dump(network, open('data/mygraph.pickle', 'wb'))
# # load graph object from file
# G = pickle.load(open('data/mygraph.pickle', 'rb'))
# m = folium.Map(location=[40, -100], zoom_start=3)
# # 标记不同的颜色
# for node in G.nodes:
#     if str(node).startswith('C'):
#         color = 'red'
#     elif str(node).startswith('T'):
#         color = 'blue'
#     else:
#         color = 'green'
#     folium.Marker(location=node.location, popup=f"Name: {node.idx}", icon=folium.Icon(color=color)).add_to(m)

# for edge in G.edges:
#     folium.PolyLine(locations=[edge[0].location, edge[1].location], color='blue').add_to(m)

# # 保存地图为 HTML 文件
# m.save('data/mygraph.pickle.html')
# print('save over')